# Portfolio Optimization


##### Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind = [];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

## Model 1

In [2]:
m1 = Model()

w1 = m1.addVars(4)

m1.addConstr(sum(w1[i]*avg_return[tickind[i]] for i in range(4)) >= 0.005)

m1.addConstr(sum(w1[i] for i in range(4)) == 1)

for i in range(4):
    m1.addConstr(w1[i] >= 0)

m1.setObjective(sum(w1[i]*w1[j]*C[tickind[i]][tickind[j]] for i in range(4) for j in range(4)), GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05


In [3]:
m1.update()
m1.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x24f4abf9
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1

## Model 2

In [4]:
m2 = Model()

w2 = m2.addVars(n)

m2.addConstr(sum(w2[i]*avg_return[i] for i in range(n)) >= 0.005)

m2.addConstr(sum(w2[i] for i in range(n)) == 1)

for i in range(n):
    m2.addConstr(w2[i] >= 0)

m2.setObjective(sum(w2[i]*w2[j]*C[i][j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

In [5]:
m2.update()
m2.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x509900e3
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3

## Model 3

In [6]:
m3 = Model()
x = m3.addVars(n,vtype = GRB.BINARY)
w3 = m3.addVars(n)
M = 100

for i in range(n):
    m3.addConstr(w3[i] >= 0)

for i in range(n):
    m3.addConstr(w3[i] <= M*x[i])

m3.addConstr(sum(x[i] for i in range(n)) <= 4)

m3.addConstr(sum(w3[i]*avg_return[i] for i in range(n)) >= 0.005)
m3.addConstr(sum(w3[i] for i in range(n)) == 1)

m3.setObjective(sum(w3[i]*w3[j]*C[i][j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

In [7]:
m3.update()
m3.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xb4812368
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

#### Question 1 a)

In [8]:
print('Optimal Objective Value:', m1.ObjVal)
print('Solver time in seconds:', m1.runtime)

for i in range(4):
    print('Weight of',tickers[tickind[i]],'is', w1[i].x)

Optimal Objective Value: 0.0001774932651657925
Solver time in seconds: 0.02113485336303711
Weight of MSFT is 0.23711749137102014
Weight of GS is 0.025859835330682878
Weight of PG is 1.5749710132192858e-09
Weight of SCHP is 0.7370226717233492


#### Question 1 b)

In [9]:
print('Optimal Objective Value:', m2.ObjVal)
print('Solver time in seconds:', m2.runtime)

Optimal Objective Value: 2.8785756399297445e-05
Solver time in seconds: 0.040003061294555664


#### Question 1 c)

In [10]:
print('Optimal Objective Value:', m3.ObjVal)
print('Solver time in seconds:', m3.runtime)

for i in range(n):
    if x[i].X:
        print('Weight of',tickers[i],'is', w3[i].X)

Optimal Objective Value: 6.753470760728118e-05
Solver time in seconds: 22.99446702003479
Weight of CME is 0.1264114192949047
Weight of LLY is 0.07547619035437456
Weight of NVDA is 0.04375370672843181
Weight of BND is 0.7543586836222889


#### Question 2 a)

Optimal risk of model 1 is higher than that of model 2 since model 2 has a much higher number of stocks, which allows for more diversification of the portfolio. Since a large number of stocks are present in model 2, there's a higher chance of minimizing risk by investing in many different stocks rather than just 4.

#### Question 2 b)

Optimal risk of model 3 is higher than that of model 2 since model 2 has a much higher number of stocks. However, the model is selecting a combination of at most 4 stocks that together produce the least risk possible. It goes through all the combinations of upto 4 stocks that when put together in a portfolio, have minimum risk.

#### Question 3 a)

In [11]:
m3a1 = Model()
xa1 = m3a1.addVars(n,vtype = GRB.BINARY)
w3a1 = m3a1.addVars(n)
M = 100

for i in range(n):
    m3a1.addConstr(w3a1[i] >= 0)

for i in range(n):
    m3a1.addConstr(w3a1[i] <= M*xa1[i])

m3a1.addConstr(sum(xa1[i] for i in range(n)) <= 4)

m3a1.addConstr(sum(w3a1[i]*avg_return[i] for i in range(n)) >= 0.005)
m3a1.addConstr(sum(w3a1[i] for i in range(n)) == 1)

m3a1.setObjective(sum(w3a1[i]*w3a1[j]*C[i][j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

m3a1.Params.TimeLimit = 10.0

m3a1.update()
m3a1.optimize()

Set parameter TimeLimit to value 10
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xb4812368
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds    

In [12]:
print('Optimal Objective Value:', m3a1.ObjVal)

Optimal Objective Value: 6.753470760728118e-05


#### We obtain the same optimal value as the original model with no time limit set. I notice that the best bound value is lower than the optimal value.

#### Question 3 b)

In [13]:
m3b = Model()
xb = m3b.addVars(n,vtype = GRB.BINARY)
w3b = m3b.addVars(n)
M = 100

for i in range(n):
    m3b.addConstr(w3b[i] >= 0)

for i in range(n):
    m3b.addConstr(w3b[i] <= M*xb[i])

m3b.addConstr(sum(xb[i] for i in range(n)) <= 4)

m3b.addConstr(sum(w3b[i]*avg_return[i] for i in range(n)) >= 0.005)
m3b.addConstr(sum(w3b[i] for i in range(n)) == 1)

m3b.setObjective(sum(w3b[i]*w3b[j]*C[i][j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

m3b.Params.MIPGap = 0.1

m3b.update()
m3b.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xb4812368
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [14]:
print('Solver time in seconds:', m3b.runtime)

Solver time in seconds: 18.36990189552307


#### The solver time is lower than that of the original model with no gap defined, since we have asked the model to stop at a defined value of gap.